<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [1]:
!pip install pandas
!pip install matplotlib

## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [2]:
# Import necessary libraries
import pandas as pd

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [3]:
# Display summary of dataset: column names, data types, non-null counts
print("=== Dataset Info ===")
print(df.info())

# Count missing values per column
print("\n=== Missing Values per Column ===")
print(df.isnull().sum())


=== Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 56.9+ MB
None

=== Missing Values per Column ===
ResponseId                 0
MainBranch                 0
Age                        0
Employment                 0
RemoteWork             10631
                       ...  
JobSatPoints_11        35992
SurveyLength            9255
SurveyEase              9199
ConvertedCompYearly    42002
JobSat                 36311
Length: 114, dtype: int64


<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [4]:
# Generate basic statistics for numerical columns
print("=== Basic Statistics for Numerical Columns ===")
print(df.describe())


=== Basic Statistics for Numerical Columns ===
         ResponseId      CompTotal       WorkExp  JobSatPoints_1  \
count  65437.000000   3.374000e+04  29658.000000    29324.000000   
mean   32719.000000  2.963841e+145     11.466957       18.581094   
std    18890.179119  5.444117e+147      9.168709       25.966221   
min        1.000000   0.000000e+00      0.000000        0.000000   
25%    16360.000000   6.000000e+04      4.000000        0.000000   
50%    32719.000000   1.100000e+05      9.000000       10.000000   
75%    49078.000000   2.500000e+05     16.000000       22.000000   
max    65437.000000  1.000000e+150     50.000000      100.000000   

       JobSatPoints_4  JobSatPoints_5  JobSatPoints_6  JobSatPoints_7  \
count    29393.000000    29411.000000    29450.000000     29448.00000   
mean         7.522140       10.060857       24.343232        22.96522   
std         18.422661       21.833836       27.089360        27.01774   
min          0.000000        0.000000        0.0

### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [5]:
# Check unique values in the 'Country' column
unique_countries = df['Country'].unique()
print("=== Unique values in 'Country' column ===")
print(unique_countries)

# Check for inconsistent or suspicious entries (e.g., typos, empty strings, NaNs)
inconsistent_entries = df['Country'].value_counts().tail(20)  # shows the least frequent 20 entries
print("\n=== Potentially inconsistent or rare entries ===")
print(inconsistent_entries)


=== Unique values in 'Country' column ===
['United States of America'
 'United Kingdom of Great Britain and Northern Ireland' 'Canada' 'Norway'
 'Uzbekistan' 'Serbia' 'Poland' 'Philippines' 'Bulgaria' 'Switzerland'
 'India' 'Germany' 'Ireland' 'Italy' 'Ukraine' 'Australia' 'Brazil'
 'Japan' 'Austria' 'Iran, Islamic Republic of...' 'France' 'Saudi Arabia'
 'Romania' 'Turkey' 'Nepal' 'Algeria' 'Sweden' 'Netherlands' 'Croatia'
 'Pakistan' 'Czech Republic' 'Republic of North Macedonia' 'Finland'
 'Slovakia' 'Russian Federation' 'Greece' 'Israel' 'Belgium' 'Mexico'
 'United Republic of Tanzania' 'Hungary' 'Argentina' 'Portugal'
 'Sri Lanka' 'Latvia' 'China' 'Singapore' 'Lebanon' 'Spain' 'South Africa'
 'Lithuania' 'Viet Nam' 'Dominican Republic' 'Indonesia' 'Kosovo'
 'Morocco' 'Taiwan' 'Georgia' 'San Marino' 'Tunisia' 'Bangladesh'
 'Nigeria' 'Liechtenstein' 'Denmark' 'Ecuador' 'Malaysia' 'Albania'
 'Azerbaijan' 'Chile' 'Ghana' 'Peru' 'Bolivia' 'Egypt' 'Luxembourg'
 'Montenegro' 'Cyprus' 'Pa

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [6]:
# Example: Standardize 'Country' column
# Create a mapping dictionary for inconsistent country names
country_mapping = {
    "United States of America": "USA",
    "United States": "USA",
    "U.S.A": "USA",
    "UK": "United Kingdom",
    "U.K.": "United Kingdom",
    "Deutschland": "Germany",
    # Add more mappings as needed
}

# Apply the mapping to the 'Country' column
df['Country'] = df['Country'].replace(country_mapping)

# Verify the changes
print("Unique countries after standardization:")
print(df['Country'].unique())

# Example: Standardize 'EdLevel' column
education_mapping = {
    "Bachelor’s degree (B.A., B.S., B.Eng., etc.)": "Bachelor's",
    "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)": "Master's",
    "Some college/university study without earning a degree": "Some College",
    "Primary/elementary school": "Primary",
    # Add more mappings as needed
}

# Apply the mapping to the 'EdLevel' column
df['EdLevel'] = df['EdLevel'].replace(education_mapping)

# Verify the changes
print("\nUnique education levels after standardization:")
print(df['EdLevel'].unique())


Unique countries after standardization:
['USA' 'United Kingdom of Great Britain and Northern Ireland' 'Canada'
 'Norway' 'Uzbekistan' 'Serbia' 'Poland' 'Philippines' 'Bulgaria'
 'Switzerland' 'India' 'Germany' 'Ireland' 'Italy' 'Ukraine' 'Australia'
 'Brazil' 'Japan' 'Austria' 'Iran, Islamic Republic of...' 'France'
 'Saudi Arabia' 'Romania' 'Turkey' 'Nepal' 'Algeria' 'Sweden'
 'Netherlands' 'Croatia' 'Pakistan' 'Czech Republic'
 'Republic of North Macedonia' 'Finland' 'Slovakia' 'Russian Federation'
 'Greece' 'Israel' 'Belgium' 'Mexico' 'United Republic of Tanzania'
 'Hungary' 'Argentina' 'Portugal' 'Sri Lanka' 'Latvia' 'China' 'Singapore'
 'Lebanon' 'Spain' 'South Africa' 'Lithuania' 'Viet Nam'
 'Dominican Republic' 'Indonesia' 'Kosovo' 'Morocco' 'Taiwan' 'Georgia'
 'San Marino' 'Tunisia' 'Bangladesh' 'Nigeria' 'Liechtenstein' 'Denmark'
 'Ecuador' 'Malaysia' 'Albania' 'Azerbaijan' 'Chile' 'Ghana' 'Peru'
 'Bolivia' 'Egypt' 'Luxembourg' 'Montenegro' 'Cyprus' 'Paraguay'
 'Kazakhstan' 'S

### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [7]:
# One-hot encode the 'Employment' column
employment_encoded = pd.get_dummies(df['Employment'], prefix='Employment')

# Display the first few rows of the encoded DataFrame
print(employment_encoded.head())

# Optionally, concatenate the encoded columns back to the original DataFrame
df = pd.concat([df, employment_encoded], axis=1)

# Verify the updated DataFrame
print(df.head())


   Employment_Employed, full-time  \
0                            True   
1                            True   
2                            True   
3                           False   
4                           False   

   Employment_Employed, full-time;Employed, part-time  \
0                                              False    
1                                              False    
2                                              False    
3                                              False    
4                                              False    

   Employment_Employed, full-time;Independent contractor, freelancer, or self-employed  \
0                                              False                                     
1                                              False                                     
2                                              False                                     
3                                              False                      

### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [8]:
# Count missing values per column
missing_values = df.isnull().sum()

# Sort columns by number of missing values in descending order
missing_values_sorted = missing_values.sort_values(ascending=False)

# Display the top 10 columns with the most missing values
print("Columns with the most missing values:")
print(missing_values_sorted.head(10))


Columns with the most missing values:
AINextMuch less integrated       64289
AINextLess integrated            63082
AINextNo change                  52939
AINextMuch more integrated       51999
EmbeddedAdmired                  48704
EmbeddedWantToWorkWith           47837
EmbeddedHaveWorkedWith           43223
ConvertedCompYearly              42002
AIToolNot interested in Using    41023
AINextMore integrated            41009
dtype: int64


<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [12]:
# Impute missing values with mean
mean_value = df['ConvertedCompYearly'].mean()
df['ConvertedCompYearly'].fillna(mean_value, inplace=True)

# Alternatively, impute with median
# median_value = df['ConvertedCompYearly'].median()
# df['ConvertedCompYearly'].fillna(median_value, inplace=True)

# Verify that missing values are handled
missing_after_imputation = df['ConvertedCompYearly'].isnull().sum()
print(f"Missing values in 'ConvertedCompYearly' after imputation: {missing_after_imputation}")


Missing values in 'ConvertedCompYearly' after imputation: 0


/tmp/ipykernel_704/2918339120.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ConvertedCompYearly'].fillna(mean_value, inplace=True)


<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [13]:
# Find the most frequent value in the 'RemoteWork' column
most_frequent_value = df['RemoteWork'].mode()[0]

# Impute missing values with the most frequent value
df['RemoteWork'] = df['RemoteWork'].fillna(most_frequent_value)

# Verify that missing values are handled
missing_after_imputation = df['RemoteWork'].isnull().sum()
print(f"Missing values in 'RemoteWork' after imputation: {missing_after_imputation}")


Missing values in 'RemoteWork' after imputation: 0


### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [15]:
# Min-Max scaling without sklearn
df['ConvertedCompYearly_MinMax'] = (df['ConvertedCompYearly'] - df['ConvertedCompYearly'].min()) / (df['ConvertedCompYearly'].max() - df['ConvertedCompYearly'].min())

# Display the first few rows to verify
print(df[['ConvertedCompYearly', 'ConvertedCompYearly_MinMax']].head())



   ConvertedCompYearly  ConvertedCompYearly_MinMax
0         86155.287263                      0.0053
1         86155.287263                      0.0053
2         86155.287263                      0.0053
3         86155.287263                      0.0053
4         86155.287263                      0.0053


<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [16]:
import numpy as np

# Add 1 to avoid log(0) if there are any zeros
df['ConvertedCompYearly_Log'] = np.log1p(df['ConvertedCompYearly'])

# Display the first few rows to verify
print(df[['ConvertedCompYearly', 'ConvertedCompYearly_Log']].head())


   ConvertedCompYearly  ConvertedCompYearly_Log
0         86155.287263                11.363918
1         86155.287263                11.363918
2         86155.287263                11.363918
3         86155.287263                11.363918
4         86155.287263                11.363918


### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [18]:
import pandas as pd
import numpy as np

# Function to convert 'YearsCodePro' to numeric
def years_to_float(years):
    if pd.isnull(years):
        return np.nan
    if years == "Less than 1 year":
        return 0.5
    elif years == "More than 50 years":
        return 51
    else:
        try:
            return float(years)
        except:
            return np.nan

# Apply conversion
df['YearsCodePro_numeric'] = df['YearsCodePro'].apply(years_to_float)

# Now categorize based on numeric years
def categorize_experience(years):
    if pd.isnull(years):
        return 'Unknown'
    elif years < 2:
        return 'Beginner'
    elif 2 <= years < 5:
        return 'Intermediate'
    elif 5 <= years < 10:
        return 'Advanced'
    else:
        return 'Expert'

df['ExperienceLevel'] = df['YearsCodePro_numeric'].apply(categorize_experience)

# Verify results
print(df[['YearsCodePro', 'YearsCodePro_numeric', 'ExperienceLevel']].head(10))


  YearsCodePro  YearsCodePro_numeric ExperienceLevel
0          NaN                   NaN         Unknown
1           17                  17.0          Expert
2           27                  27.0          Expert
3          NaN                   NaN         Unknown
4          NaN                   NaN         Unknown
5          NaN                   NaN         Unknown
6            7                   7.0        Advanced
7          NaN                   NaN         Unknown
8          NaN                   NaN         Unknown
9           11                  11.0          Expert


### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
